<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://files.oproject.org/tmvalogo.png" height="50%" width="50%">

# TMVA  Reader Example 

#### Example of applying the trained classified of TMVA on a data set and evaluate the classifier

In [1]:
TMVA::Tools::Instance();

### Choose TMVA method to evaluate

In [2]:
std::vector<TString> methodNames = {"BDT","DL_CPU","Keras_dense13"}

(std::vector<TString> &) { @0x7fb45fd773f0, @0x7fb45fd77408, @0x7fb45fd77420 }


###  Give input file 


In [3]:
TFile * inputFile = TFile::Open("http://www.cern.ch/moneta/root/Higgs_data_200k.root");

In [4]:
inputFile->ls();


TWebFile**		http://www.cern.ch/moneta/root/Higgs_data_200k.root	
 TWebFile*		http://www.cern.ch/moneta/root/Higgs_data_200k.root	
  KEY: TTree	sig_tree;1	Signal Tree
  KEY: TTree	bkg_tree;1	Background Tree
  KEY: TTree	test_tree;1	Testing Tree


In [5]:
auto sig_tree = (TTree*) inputFile->Get("sig_tree");
auto bkg_tree = (TTree*) inputFile->Get("bkg_tree");
sig_tree->Print(); 

******************************************************************************
*Tree    :sig_tree  : Signal Tree                                            *
*Entries :   200000 : Total =        22485706 bytes  File  Size =   19872183 *
*        :          : Tree compression factor =   1.13                       *
******************************************************************************
*Br    0 :lepton_pT : Float_t F                                              *
*Entries :   200000 : Total  Size=     803071 bytes  File Size  =     756746 *
*Baskets :       25 : Basket Size=      32000 bytes  Compression=   1.06     *
*............................................................................*
*Br    1 :lepton_eta : Float_t F                                             *
*Entries :   200000 : Total  Size=     803099 bytes  File Size  =     739610 *
*Baskets :       25 : Basket Size=      32000 bytes  Compression=   1.08     *
*...................................................

### Declare Reader


In [6]:

TMVA::Reader reader( "!Color:!Silent" );


### Add Variable

We add variables to the reader exactly in the same way we did for the **DataLoader** during the training

We need to specify the address of the variable in order to pass it to TMVA when we iterate on the TTree


In [7]:
float m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb;
float jet1_pt, jet2_pt, jet3_pt, jet4_pt,  lepton_pT, missing_energy_magnitude;

In [8]:
// add variables 
reader.AddVariable("m_jj",&m_jj);
reader.AddVariable("m_jjj",&m_jjj);
reader.AddVariable("m_lv",&m_lv);
reader.AddVariable("m_jlv",&m_jlv);
reader.AddVariable("m_bb",&m_bb);
reader.AddVariable("m_wbb",&m_wbb);
reader.AddVariable("m_wwbb",&m_wwbb);

reader.AddVariable("jet1_pt",&jet1_pt);
reader.AddVariable("jet2_pt",&jet2_pt);
reader.AddVariable("jet3_pt",&jet3_pt);
reader.AddVariable("jet4_pt",&jet4_pt);

reader.AddVariable("lepton_pT",&lepton_pT);
reader.AddVariable("missing_energy_magnitude",&missing_energy_magnitude);


## Setup Classifiers

We set up the classifiers by reading the input weights from the appropriate files

The file is stored for example as 

####  *dataset/weights/TMVAClassification_BDT.weights.xml

In [9]:
for (size_t i = 0; i < methodNames.size(); ++i) { 
   TString weightfile = "dataset/weights/TMVA_Higgs_Classification_" + methodNames[i] + ".weights.xml";
   reader.BookMVA( methodNames[i], weightfile );
}

                         : Booking "BDT" of type "BDT" from dataset/weights/TMVA_Higgs_Classification_BDT.weights.xml.
                         : Reading weight file: dataset/weights/TMVA_Higgs_Classification_BDT.weights.xml
<HEADER> DataSetInfo              : [Default] : Added class "Signal"
<HEADER> DataSetInfo              : [Default] : Added class "Background"
                         : Booked classifier "BDT" of type: "BDT"
                         : Booking "DL_CPU" of type "DL" from dataset/weights/TMVA_Higgs_Classification_DL_CPU.weights.xml.
                         : Reading weight file: dataset/weights/TMVA_Higgs_Classification_DL_CPU.weights.xml
                         : Booked classifier "DL_CPU" of type: "DL"
                         : Booking "Keras_dense13" of type "PyKeras" from dataset/weights/TMVA_Higgs_Classification_Keras_dense13.weights.xml.
<FATAL>                          : Unknown method in map: PyKeras
***> abort program execution


### Book histogram to visualize the response

In [ ]:
auto h1 = new TH1D("h1","Classifier Output on Background Events",100,-1,1);
auto h2 = new TH1D("h2","Classifier Output on Signal Events",100,-1,1);
std::vector<std::vector<float>> vout(methodNames.size());

## Iterate on the events from the input TTree 

In [ ]:
auto readEvents = [&](TTree & tree, TH1 & histo) {
  tree.SetBranchAddress("m_jj",&m_jj);
  tree.SetBranchAddress("m_jjj",&m_jjj);
  tree.SetBranchAddress("m_lv",&m_lv) ;
  tree.SetBranchAddress("m_jlv",&m_jlv);
  tree.SetBranchAddress("m_bb",&m_bb);
  tree.SetBranchAddress("m_wbb",&m_wbb);
  tree.SetBranchAddress("m_wwbb",&m_wwbb);
    
  tree.SetBranchAddress("jet1_pt",&jet1_pt);
tree.SetBranchAddress("jet2_pt",&jet2_pt);
tree.SetBranchAddress("jet3_pt",&jet3_pt);
tree.SetBranchAddress("jet4_pt",&jet4_pt);

tree.SetBranchAddress("lepton_pT",&lepton_pT);
tree.SetBranchAddress("missing_energy_magnitude",&missing_energy_magnitude);


    
  for (size_t i = 0; i < methodNames.size(); ++i ) {
    vout[i].clear(); 
    vout[i].resize(tree.GetEntries());
  }
  for (Long64_t ievt=0; ievt < tree.GetEntries();ievt++) {

      if (ievt%500 == 0) std::cout << "--- ... Processing event: " << ievt << std::endl;

      tree.GetEntry(ievt);
      
      for (size_t i = 0; i < methodNames.size(); ++i ) {
    
         auto output = reader.EvaluateMVA(methodNames[i]);
  
         vout[i][ievt] = output;  
      }
  }
};
    

### Evaluation of the classifier on the given input events

We iterate on the input event in the given TTree. We provide as input first the background tree 
We need first to define the branches we are reading from the trees and then we can read the events calling 
the readEvents function we have defined before

In [ ]:
// read events and fill histogram
readEvents(*bkg_tree,*h1); 

In [ ]:
std::vector<std::vector<float>> bkg_results = vout; 

### Evaluation on the signal  events

We evaluate now the classifier on the another tree (e.g. the signal events)



In [ ]:
readEvents(*sig_tree,*h2); 

In [ ]:
std::vector<std::vector<float>> sig_results = vout; 

## Make ROC plot

In [ ]:
for (size_t i = 0; i < methodNames.size(); ++i) {
    TMVA::ROCCurve * roc = new TMVA::ROCCurve(sig_results[i], bkg_results[i]);
    TGraph *  roc_graph  = roc->GetROCCurve(); 
    roc_graph->SetTitle(methodNames[i]);
    std::cout << "method " << methodNames[i] << " ROC integral = " << roc->GetROCIntegral() << std::end;
    roc_graph->SetLineColor(i+1);
    if (i == 0) roc_graph->Draw("AL");
    else roc_graph->Draw("L");
}

In [ ]:
gPad->Draw();
std::cout << "ROC integral = " << roc.GetROCIntegral(); 